# Visualizing Changes In Opioid Treatment Policy

*Please note: This notebook uses open access data*

#### J Montgomery Maxwell

In [ ]:
import pandas as pd
import numpy as np
import gen3
import openpyxl
import plotly.express as px
import requests
import json

### Define Data Mappings

Dictionary for mapping the names of States and Territories to their respective abbreviations. Create additional mappings for cleaning data.


In [ ]:
state_abbrev = {
    "Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR", "California": "CA", "Colorado": "CO", 
    "Connecticut": "CT", "Delaware": "DE", "Florida": "FL", "Georgia": "GA", "Hawaii": "HI", "Idaho": "ID",
    "Illinois": "IL", "Indiana": "IN", "Iowa": "IA", "Kansas": "KS", "Kentucky": "KY", "Louisiana": "LA",
    "Maine": "ME", "Maryland": "MD", "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN", 
    "Mississippi": "MS", "Missouri": "MO", "Montana": "MT", "Nebraska": "NE", "Nevada": "NV",
    "New Hampshire": "NH", "New Jersey": "NJ", "New Mexico": "NM", "New York": "NY", "North Carolina": "NC", 
    "North Dakota": "ND", "Ohio": "OH",  "Oklahoma": "OK", "Oregon": "OR", "Pennsylvania": "PA", 
    "Rhode Island": "RI", "South Carolina": "SC", "South Dakota": "SD", "Tennessee": "TN", "Texas": "TX", 
    "Utah": "UT", "Vermont": "VT", "Virginia": "VA", "Washington": "WA", "West Virginia": "WV", 
    "Wisconsin": "WI", "Wyoming": "WY", "District of Columbia": "DC", "American Samoa": "AS", "Guam": "GU", 
    "Northern Mariana Islands": "MP", "Puerto Rico": "PR", "United States Minor Outlying Islands": "UM", 
    "U.S. Virgin Islands": "VI"}

policyMap = {0: 'Not Approved', 1:'Approved'}
waiverMap = {'.': 0, 0:0, 1:1}

### Import And Clean Data

Import data using the Gen3 Python SDK. Importing three data files and their respective codebooks.

In [ ]:
!gen3 drs-pull object dg.6VTS/5200158e-e9fe-44ef-96c9-e89ecd402fc4
!gen3 drs-pull object dg.6VTS/2b83e419-8d3d-4569-b9a1-a52ecd387cba
!gen3 drs-pull object dg.6VTS/a0a8785a-8663-47b9-95ea-a1813612a2f1
!gen3 drs-pull object dg.6VTS/abe9cd49-fc86-4c9b-b9d0-f8c0280d8aaa
!gen3 drs-pull object dg.6VTS/b7974ffe-2e46-47cf-9d57-4d8900d7a40f
!gen3 drs-pull object dg.6VTS/dca15d95-aac5-4879-88cb-3a740398f26c

Here we look at three key features for U.S. States. The code name for the policy is listed at the end of each line. 
- Has the state approved telehealth MOUD treatment with buprenorphine or methadone for existing opioid treatment  patients? (Telehealth Treatment)
- Has the state received an approved section 1115 Medicaid waiver that explicitely address SUD treatment? (SUD Medicaid Waiver)
- Have state correctional facilities approved modified policies explicitly related to MOUD treatment during COVID-19? (Incarceration MOUD Treatment)

In [ ]:
df1 = pd.read_excel('buprenorphine-and-methadone-during-covid-19-data-020222.xlsx')
df2 = pd.read_excel('covid-19-state-medicaid-waivers-data-020222.xlsx')
df3 = pd.read_excel('covid-19-moud-at-state-correctional-facilities-data-020222.xlsx')

df1['Policy Change'] = ((df1['Telehealth_existing_bup'] + df1['Telehealth_existing_methadone']) != 0).astype(int)
df1['Policy'] = 'Telehealth MOUD Treatment'
df1 = df1[['Jurisdictions', 'Policy Change', 'Policy']]

df2['Policy Change'] = df2['JM_15sud'].map(waiverMap)
df2['Policy'] = 'SUD Medicaid Waiver'
df2 = df2[['Jurisdictions', 'Policy Change', 'Policy']]

df3['Policy Change'] = df3['JC_MOUDlaw']
df3['Policy'] = 'MOUD Treatment While Incarcerated'
df3 = df3[['Jurisdictions', 'Policy Change', 'Policy']]

df = pd.concat([df1, df2, df3],ignore_index=True)
df['Abbreviations'] = df['Jurisdictions'].map(state_abbrev)
df['Change In Policy'] = df['Policy Change'].map(policyMap)
df = df[['Jurisdictions', 'Abbreviations', 'Change In Policy', 'Policy']]
df

## Single & Multi Feature Choropleth Maps

You can create choropleth maps using the plotly.express.choropleth package. 

Both discrete and continuous data can be represented in these plots. 

In [ ]:
fig = px.choropleth(df[df['Policy'] == 'Telehealth MOUD Treatment'], locations='Abbreviations', locationmode="USA-states", 
                    color='Change In Policy', color_discrete_map={'Not Approved':'Gray', 'Approved':'Purple'}, 
                    scope='usa', title='Telehealth MOUD Treatment With Buprenorphine Or Methadone')
fig.show()

You can show multiple features or the same feature over a period of time using the 'animation_frame' argument. Here we create an interactive plot which allows us to toggle between the three changes in public health policy which we are investigating. 

In [ ]:
fig = px.choropleth(df, locations='Abbreviations', locationmode="USA-states", color='Change In Policy', 
                    color_discrete_map={'Not Approved':'Gray', 'Approved':'Purple'}, animation_frame='Policy', 
                    scope='usa', title='Changes In Opioid Treatment Policy During COVID-19')
fig.show()